In [5]:
import os
from PIL import Image, ImageDraw
import numpy as np
from collections import deque

def load_maze_image(image_path):
    return Image.open(image_path).convert("RGB")

def image_to_grid(image, invert=False):
    pixels = np.array(image)
    white = (pixels[:, :, 0] > 200) & (pixels[:, :, 1] > 200) & (pixels[:, :, 2] > 200)
    grid = white.astype(int)
    return 1 - grid if invert else grid

def find_start_end(grid):
    h, w = grid.shape
    border_coords = []

    # Top row (right to left)
    for x in range(w-1, -1, -1):
        border_coords.append((0, x))

    # Left column (top to bottom)
    for y in range(1, h):
        border_coords.append((y, 0))

    # Bottom row (left to right)
    for x in range(1, w):
        border_coords.append((h-1, x))

    # Right column (bottom to top)
    for y in range(h-2, 0, -1):
        border_coords.append((y, w-1))

    white_batches = []
    batch = []

    for y, x in border_coords:
        if grid[y, x] == 1:
            batch.append((y, x))
        elif batch:
            if len(batch) > 1:
                white_batches.append(batch)
            batch = []

    if batch and len(batch) > 1:
        white_batches.append(batch)

    if len(white_batches) >= 2:
        start = white_batches[0][len(white_batches[0]) // 2]
        end = white_batches[1][len(white_batches[1]) // 2]
        return start, end

    raise ValueError("Could not find two valid entrance/exit points.")

def bfs(grid, start, end):
    h, w = grid.shape
    visited = np.zeros_like(grid, dtype=bool)
    parent = {}

    q = deque([start])
    visited[start] = True

    while q:
        y, x = q.popleft()
        if (y, x) == end:
            break

        for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:
            ny, nx = y + dy, x + dx
            if 0 <= ny < h and 0 <= nx < w and not visited[ny, nx] and grid[ny, nx] == 1:
                visited[ny, nx] = True
                parent[(ny, nx)] = (y, x)
                q.append((ny, nx))

    return parent

def reconstruct_path(parent, start, end):
    path = []
    current = end
    while current != start:
        path.append(current)
        if current not in parent:
            return []  # No path found
        current = parent[current]
    path.append(start)
    path.reverse()
    return path

def draw_path(image, path, color=(255, 0, 0)):
    draw = ImageDraw.Draw(image)
    for y, x in path:
        draw.point((x, y), fill=color)
    return image

def solve_and_save(image_path, save_path):
    image = load_maze_image(image_path)

    for invert in [False, True]:
        grid = image_to_grid(image, invert=invert)
        try:
            start, end = find_start_end(grid)
            parent = bfs(grid, start, end)
            path = reconstruct_path(parent, start, end)
            if path:
                solved_image = draw_path(image, path)
                solved_image.save(save_path)
                return
        except ValueError:
            continue
    print(f"Start/end not found or no path: {image_path}")

# ---- Loop over all images ----
input_folder = "data"
output_folder = "data-solved"

os.makedirs(output_folder, exist_ok=True)

for fname in os.listdir(input_folder):
    if fname.endswith(".png"):
        in_path = os.path.join(input_folder, fname)
        out_path = os.path.join(output_folder, fname)
        solve_and_save(in_path, out_path)


